In [2]:
import os

In [3]:
reference_fasta_file = "/mnt/projects/pasteur_bacterias/refs/Salmon1418_reference.fna"
# reference_fasta_file = "/mnt/projects/pasteur_bacterias/refs/Ecoli2tube_K12.fasta"

In [4]:
fastq1_file = "/mnt/projects/pasteur_bacterias/"
fastq2_file = "/mnt/projects/pasteur_bacterias/"

In [5]:
folder_with_references = "/mnt/projects/pasteur_bacterias/refs/"
if not os.path.isdir(folder_with_references):
    os.makedirs(folder_with_references)

In [6]:
%%bash
ls /mnt/projects/pasteur_bacterias/refs/ -hlrt

total 30M
-rw-rw----+ 1 mmenshenina mmenshenina 4.7M Jun  9 11:21 Styphi92.reference.fasta
-rw-r--r--  1 dzilov      dzilov      4.8M Jun  9 11:23 Salmon1418_reference.fna
-rw-rw-r--  1 sochkalova  sochkalova  4.7M Jun  9 11:26 salmon_ref.fasta
-rw-rw-r--  1 vdikaya     vdikaya     4.7M Jun  9 11:29 salmon13.fasta
-rw-rw-r--  1 pchesnokova pchesnokova 4.5M Jun  9 11:31 Ecoli2tube_K12.fasta
-rw-rw-r--  1 sochkalova  sochkalova  4.7M Jun  9 12:51 salmon1502.fasta
-rw-r--r--  1 sryaxovsky  sryaxovsky  1.5M Jun 10 11:31 Styphy87_ref_assembly.fasta
-rw-r--r--  1 sryaxovsky  sryaxovsky   339 Jun 10 11:31 report.txt
drwxrwxr-x  2 sryaxovsky  sryaxovsky  4.0K Jun 15 12:33 ncbi-genomes-2020-06-10


In [7]:
def sc_iter_fasta_for_refseq(file_name):
    """ Iter over fasta file."""

    header = None
    seq = []
    with open(file_name) as fh:
        data = fh.readlines()
        for line in data:
            if line.startswith(">"):
                if seq:
                    sequence = "".join(seq).upper()
                    yield (header, sequence)
                header = line.strip()[1:]
                seq = []
                continue
            seq.append(line.upper().strip())
        if seq or header:
            sequence = "".join(seq).upper()
            yield (header, sequence)

In [8]:

for header, sequence in sc_iter_fasta_for_refseq(reference_fasta_file):
    print(header, len(sequence))
    break
    
len(sequence)

AE006468.2 Salmonella enterica subsp. enterica serovar Typhimurium str. LT2, complete genome 4857450


4857450

In [9]:
def print_next_R(kmer, kmer2freq, cutoff=50):
    ''' Returns
    - R as a dictionary nucleotide to tf
    '''
    R = {}
    a = kmer2freq[kmer[1:]+'A']
    if a and a > cutoff:
        R["A"] = a
    c = kmer2freq[kmer[1:]+'C']
    if c and c > cutoff:
        R["C"] = c
    t = kmer2freq[kmer[1:]+'T']
    if t and t > cutoff:
        R["T"] = t
    g = kmer2freq[kmer[1:]+'G']
    if g and g > cutoff:
        R["G"] = g
    return R


def print_prev_R(kmer, kmer2freq, cutoff=0):
    ''' Returns:
    - R as a dictionary nucleotide to tf
    '''
    R = {}
    a = kmer2freq['A'+kmer[:-1]]
    if a and a > cutoff:
        R["A"] = a
    c = kmer2freq['C'+kmer[:-1]]
    if c and c > cutoff:
        R["C"] = c
    t = kmer2freq['T'+kmer[:-1]]
    if t and t > cutoff:
        R["T"] = t
    g = kmer2freq['G'+kmer[:-1]]
    if g and g > cutoff:
        R["G"] = g
    return R

In [31]:
graph = get_graph(sequence, 21)

In [40]:
from collections import defaultdict

### 101 illumina 
### 350 PE illumina 101

def get_graph(sequence, k):
    graph = defaultdict(int)
    for i in range(len(sequence)-k+1):
        kmer = sequence[i:i+k]
        graph[kmer] += 1
    return graph

In [55]:
number = 23
fork_number = []
contig_number = []
while i < 3:
    contig_number.append(number)
    graph = get_graph(sequence, number)
    
    for kmer,tf in graph.items():
        if tf > 1:
            fork_number.append(number)
        else:
            number = number // 2
    i+=1

print(one_contig)
print(fork_number)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
def get_right_forks(kmer, graph):
    ''' Returns
    - R as a dictionary nucleotide to tf
    '''
    forks = {}
    a = graph[kmer[1:]+'A']
    if a:
        forks["A"] = a
    c = graph[kmer[1:]+'C']
    if c:
        forks["C"] = c
    t = graph[kmer[1:]+'T']
    if t:
        forks["T"] = t
    g = graph[kmer[1:]+'G']
    if g:
        forks["G"] = g
    return forks


def print_prev_R(kmer, kmer2freq, cutoff=0):
    ''' Returns:
    - R as a dictionary nucleotide to tf
    '''
    R = {}
    a = kmer2freq['A'+kmer[:-1]]
    if a and a > cutoff:
        R["A"] = a
    c = kmer2freq['C'+kmer[:-1]]
    if c and c > cutoff:
        R["C"] = c
    t = kmer2freq['T'+kmer[:-1]]
    if t and t > cutoff:
        R["T"] = t
    g = kmer2freq['G'+kmer[:-1]]
    if g and g > cutoff:
        R["G"] = g
    return R

In [23]:
kmer = "GTTA"
for nucl,tf in get_right_forks(kmer, graph).items():
    print(nucl, kmer[1:]+nucl, tf)

A TTAA 21339
C TTAC 18705
T TTAT 24242
G TTAG 8045


In [24]:
# my_assembler 
def UBER_ASSEMBLER(sequence, graph, k, verbose=False):
    ''' Stupid assembler.
    '''
    start_kmer = sequence[:k]
    queue = [start_kmer]
    contigs = []
    seen_kmers = set()
    while queue:
        kmer = queue.pop(0)
        contig = kmer
        ### handle repeats
        if kmer in seen_kmers:
            continue
        seen_kmers.add(kmer)
        ### assembly kmer until fork or end
        while True:
            R = get_right_forks(kmer, graph)
            if len(R) == 1:
                contig += list(R.keys())[0]
                kmer = kmer[1:] + list(R.keys())[0]
            elif len(R) > 1:
                for nucl in R:
                    rkmer = kmer[1:] + nucl
                    queue.append(rkmer)
                break
            elif len(R) == 0:
                break
        contigs.append(contig)
        lengths = [len(x) for x in contigs]
        if verbose:
            print(len(contigs), get_n50(lengths) )
    if verbose:
        print("DONE")
    return contigs

In [25]:
'''
Salmonella:
Illimina SE101bp = 156, 178804
Illimina SE350bp = 50, 383101
Length = 4857450

E. coli:
Illimina SE101bp = 261, 133750
Illimina SE350bp = 97, 179619
Length = 4641652

'''
#

'\nSalmonella:\nIllimina SE101bp = 156, 178804\nIllimina SE350bp = 50, 383101\nLength = 4857450\n\nE. coli:\nIllimina SE101bp = 261, 133750\nIllimina SE350bp = 97, 179619\nLength = 4641652\n\n'

In [34]:
UBER_ASSEMBLER(sequence,graph, 21)

['AGAGATTACGTCTGGTTGCAAGAGATCATGACAGGGGGAATTGGTTGAAAATAAATATATCGCCAGCAGCACATGAACAAGTTTCGGAATGTGATCAATTTAAAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAATACAAGACAGACAAATAAAAATGACAGAGTACACAACATCCATGAACCGCATCAGCACCACCACCATTACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGAACAGTGCGGGCTTTTTTTTCGACCAGAGATCACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAAGGGCAGGTAGCGACCGTACTTTCCGCCCCCGCGAAAATTACCAACCATCTGGTGGCAATGATTGAAAAAACTATCGGCGGCCAGGATGCTTTGCCGAATATCAGCGATGCAGAACGTATTTTTTCTGACCTGCTCGCAGGACTTGCCAGCGCGCAGCCGGGATTCCCGCTTGCACGGTTGAAAATGGTTGTCGAACAAGAATTCGCTCAGATCAAACATGTTCTGCATGGTATCAGCCTGCTGGGTCAGTGCCCGGATAGCATCAACGCCGCGCTGATTTGCCGTGGCGAAAAAATGTCGATCGCGATTATGGCGGGACTTCTGGAGGCGCGTGGGCATCGCGTCACGGTGATCGATCCGGTAGAAAAATTGCTGGCGGTGGGCCATTACCTTGAATCTACCGTTGATATCGCGGAATCGACTCGCCGTATCGCCGCCAGCCAGATCCCGGCCGATCACATGATCCTGATGGCGGGCTTTACCGCCGGTAATGAAAAGGGTGAACTGGTGGTGCTGGGCCGTAATGGTTCCGACTATTCCGCCGCCGTGCTGGCCGCCTGTTTACGCGCTGA

In [35]:
lengths = [len(x) for x in contigs]
len(contigs), lengths, get_n50(lengths)

NameError: name 'contigs' is not defined

In [36]:
def get_n50(lengths, ref=None):
    """ Get (N50 contig length, N50, shortest contig, longest contig) statistics for list of contigs lengths.
    @param lengths: a list of contig lengths
    @return (N50, L50, shortest contig, longest contig)
    """
    if not lengths:
        return 0, 0, 0, 0
    lengths.sort(reverse=True)
    if ref:
        total = ref
    else:
        total = sum(lengths)
    n50 = 0
    l50 = 0
    shortest_seq = min(lengths)
    longest_seq = max(lengths)
    for x in lengths:
        l50 += 1
        n50 += x
        if n50 >= total/2:
            return x


In [37]:
def my_binary_search (a):
    mid = len(a) // 2
    value = a[mid]

    low = 0
    high = len(a) - 1

    while a[mid] != value and low <= high:
        if value > a[mid]:
            low = mid + 1
        else:
            high = mid - 1
        mid = (low + high) // 2

    if low > high:
        print("No value")
    else:
        print("ID =", mid, "value=",value)
        return mid,value


In [38]:
#search in 1000
def generate_list_k():
    list_k = []
    i =23
    frame=1000
    list_k.append(i)
    while i < 8000:
        i+=frame
        list_k.append(i)
    return list_k


def generate_1_fork(list_k):
    #добиться вилки
    while len(list_k)!= 2:
        ID,k = my_binary_search (list_k)
        print(ID,k)

        print("Build graph")
        graph = get_graph(sequence, k)
        print("Assembly")
        contigs = UBER_ASSEMBLER(sequence, graph, k)
        print("Get stats")
        lengths = [len(x) for x in contigs]

        print(k, len(contigs), lengths, get_n50(lengths))
        if len(contigs) ==1:
            list_k = list_k[:ID+1]
        else:
            list_k = list_k[ID:]#потому что так устроены питоновские списки.можно его не включать но тогда вилка забаггуется
            list_k.append((list_k[-1]+frame))# если не попали в contig= 1 вообще. потом этот костыль убирается

    print('final fork for assembly: ', list_k, 'my choose:', list_k[1]) 
    return list_k

# any level assembly
def lvl_assembly(list_k, frame):
    start= list_k[0]
    final=  list_k[1]
    list_k=[]
    list_k.append(start)
    while start < final:
        start+=frame
        list_k.append(start) 
    return list_k
def generate_any_fork(list_k):
    #добиться вилки
    while len(list_k)!= 2:
        ID,k = my_binary_search (list_k)
        print(ID,k)

        print("Build graph")
        graph = get_graph(sequence, k)
        print("Assembly")
        contigs = UBER_ASSEMBLER(sequence, graph, k)
        print("Get stats")
        lengths = [len(x) for x in contigs]

        print(k, len(contigs), lengths, get_n50(lengths))
        if len(contigs) ==1:
            list_k = list_k[:ID+1]
        else:
            list_k = list_k[ID:]#потому что так устроены питоновские списки
            
    print('final fork for assembly: ', list_k, 'my choose:', list_k[1]) 
    return list_k
  
    
    
# except IndexError:
#     print("kavkavkakvavakvkakvakvakkavkakvakakvkavkav")
#     k +=1000
#     print("Build graph")
#     graph = get_graph(sequence, k)
#     print("Assembly")
#     contigs = UBER_ASSEMBLER(sequence, graph, k)
#     print("Get stats")
#     lengths = [len(x) for x in contigs]
#     list_lenghts.append(get_n50(lengths))
#     print(len(contigs), lengths, get_n50(lengths))
    

In [39]:
#обертка 
print('choose lvl assembly. pick from 1 to 4, than more a choosen number, by that better assembly and longer in time')
lvl = int(input())
#генерим
list_k = generate_list_k()
#генерация 1 основной гибкой вилки    
list_k = generate_1_fork(list_k)
if lvl == 1:
    print('that is all')
elif lvl == 2:
    # lxl assembly должен на 100 принять
    frame =100
    list_k = lvl_assembly(list_k, frame)
    generate_any_fork(list_k)
elif lvl == 3:
        # lxl assembly должен на 100 принять
    frame =100
    list_k = lvl_assembly(list_k, frame)
    generate_any_fork(list_k)
    frame =10
    list_k = lvl_assembly(list_k, frame)
    generate_any_fork(list_k)
elif lvl == 4:
    frame =100
    list_k = lvl_assembly(list_k, frame)
    generate_any_fork(list_k)
    frame =10
    list_k = lvl_assembly(list_k, frame)
    generate_any_fork(list_k)
    frame =1
    list_k = lvl_assembly(list_k, frame)
    generate_any_fork(list_k)



choose lvl assembly. pick from 1 to 4, than more a choosen number, by that better assembly and longer in time
4
ID = 4 value= 4023
4 4023
Build graph
Assembly
Get stats
4023 3 [2353586, 1668963, 842945] 1668963


NameError: name 'frame' is not defined

In [ ]:
import aindex

settings = {
    "index_prefix": "/mnt/projects/pasteur_bacterias/SRR9163323/SRR9163323_1.fastq.23",
}

graph = aindex.load_aindex(settings, skip_aindex=True, skip_reads=True)

In [ ]:
### assembly from raw reads
k = 23
contigs = UBER_ASSEMBLER(sequence, graph, k, verbose=True)
print("Get stats")
lengths = [len(x) for x in contigs]
print(len(contigs), lengths, get_n50(lengths))

In [32]:
print('kek')

kek
